<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

# Ein Machine-Learning-Projekt vonAbisZ
<h4>1. Betrachte das Gesamtbild</h4>
<h4>2. Beschaffe die Daten</h4>
<h4>3. Erkunde und visualisiere die Daten</h4>
<h4>4. Bereite die Daten für Machine-Learning-Algorithmen vor</h4>
<h4>5. Wähle ein Modell aus und trainiere es</h4>
<h4>6. Optimiere das Modell</h4>
<h4>7. Nimm das System in Betrieb, überwache und warte es</h4>

# PKW-Preise schätzen

In diesem Arbeitsblatt wollen wir eine auf Grundlage von Daten, die auf einem Online-Marktplatz für Gebrauchtfahrzeuge [AutoScout24](https://www.autoscout24.de) gesammelt wurden, ein Modell zur Preisvorhersage für PKWs erstellen. Da unsere Zielvariable - der Preis - ein quantitatives Merkmal ist, führen wir eine Regressionsanalyse durch. Die Regression ist ein statistisches Verfahren, mit dem man eine beobachtete, abhängige Variable durch eine oder mehrere unabhängige Variablen zu erklären versucht.
Die **Lineare Regression** ist dabei ein Sonderfall der Regressionsanalyse, bei dem zur Beschreibung der Daten ein lineares Modell herangezogen wird. Das bedeutet, die abhängige Variable wird als Linearkombination der (mit den Regressionskoeffizienten) gewichteten unabhängigen Variablen beschrieben.

Die verwendeten Daten wurden im Mai 2015 für ein Projekt an der ETH Zürich gesammelt.
Der Datensatz liegt als CSV (comma-separated values) Datei vor, einem gängigen Format für tabellarische Daten.
Wir benutzen die Python Bibliothek **Pandas**, um den Datensatz einzulesen und um ihn zu verarbeiten.
Die Funktion `read_csv` importiert eine CSV-Datei in ein `DataFrame` Objekt.
Da der Datensatz einige nicht-ASCII Zeichen enthält, verwenden wir das ISO-8859-1 (Latin-1) Zeichenformat beim Import.
Mit `head(10)` geben wir die ersten 10 Zeilen des Datensatzes aus.

In [ ]:
import numpy as np
import pandas as pd
import os
import tarfile
import urllib.request

url = "https://github.com/fh-swf-hgi/ml/raw/main/u2/vw_golf_2015_complete.csv"
dfile = "./vw_golf_2015_complete.csv"

if not os.path.isfile(dfile):
    urllib.request.urlretrieve(url, dfile)
    
df = pd.read_csv("vw_golf_2015_complete.csv", encoding = "ISO-8859-1")
df.head(10)

Einige weitere Informationen, z.B. die Namen und Datentypen der Spalten, erhalten wir mit der Funktion `info()`:

In [ ]:
df.info()

Mit `describe()`erhalten wir einige Statistik-Informationen über die Spalten des Datensatzes:

In [ ]:
df.describe()

Die Datei enthält mit ca. 70.000 Datenpunkten recht viele Einträge.
Für unser Beispiel wollen wir den Datensatz etwas reduzieren.
Dazu wählen wir ein spezielles Automodell aus.
In der folgenden Zelle werden die Spalten *marke* und *modell* ausgewählt und in der resultierenden Tabelle alle doppelten Einträge verworfen. Wir erhalten so einen Dataframe der alle Modelltypen einmalig enthält.

In [ ]:
df_typen = df[["marke", "modell"]].drop_duplicates()
df_typen.info()

Die Neue Tabelle enthält 705 unterschiedliche Typen.
Da die Indizes aus der ursprünglichen Tabelle übernommen werden, indizieren wir den neuen Datensatz neu durch.
Von dem resultierenden Dataframe geben wir die ersten 10 Zeilen aus.

In [ ]:
df_typen.reset_index(drop=True, inplace=True)
df_typen.loc[:9]

Mit `query("modell=='Golf VI'")` wählen wir die Golf-VI Modelle aus der Liste aus und speichern die Datenpunkte in einen neuen `DataFrame`.

In [ ]:
df_golf6 = df.query("modell=='Golf VI'")
df_golf6.info()

## Kategoriale Merkmale umwandeln

Unser Datensatz enthält mehrere Spalten (Merkmale) deren Werte keine "Zahlen" sondern Zeichenketten sind.
In der obigen Ausgabe des DataFrames erkennt man diese Merkmale man Datentyp (`Dtype`) `object`.
Diese sogenannten **kategorialen Variablen** können Zeichenketten oder auch numerische Werte besitzen (beispielsweise 0 = männlich und 1 = weiblich) und kommen in Datensätzen recht häufig vor.
Für Datenanalysen stellen kategoriale (oder *qualitative*) Variable ein gewisses Problem dar, die verwendeten statistischen Methoden erfordern es normalerweise, dass mit Merkmalen *gerechnet* wird und hierzu ist ein Zahlenwert für die einzelnen Merkmale notwendig.

Man könnte auf die Idee kommen, einfach alle vorkommenden Werte einer kategorialen Variable durch zu nummerieren.
Dies ist aber nur dann möglich, wenn die kategoriale Variable *ordinal* ist.
Bei ordinalen Merkamlen lassen sich die Werte der Variablen in eine natürliche Reihenfolge bringen (z. B. Schulnoten von "sehr gut" bis "ungenügend" oder Dienstränge beim Militär).
Ordnet man diesen Kategorieen entsprechende Werte zu, so bleibt die Reihenfolge in durch die Wertigkeiten erhalten.
Häufig sind kategoriale Variable aber *nominal*, d.h. eis gibt keine natürliche Reihenfolge der Werte (z.B. Personennamen oder Steuerklassen).
Nummeriert man die Werte eine nominalen Variablen einfach durch, führt dies in der Regel dazu, dass statistische Verfahren diese "Ordnung" fehlinterpretieren und damit zu falschen Schlussfolgerungen kommen.

Wir müssen also in unserem Datensatz die kategorialen Variablen entweder in metrische Variablen überführen oder bei unseren Analysen ausblenden.
Hierfür gibt es verschiedene Methoden, die wir noch näher betrachten werden.
An dieser Stelle wollen wir eine einfache, aber in der Praxis recht häufig verwendete Technik verwenden, nämlich die sogenannte *One-Hot-Codierung*.
Dabei werden die $N$ unterschiedlichen Werte eines Merkmals durch $N$ separate neue Merkmale $m_1, m_2, \dots, m_N$ dargestellt.
Besitzt ein Datenpunkt bei dem ursprünglichen Merkmal den Wert $y$, so trägt nur das neue Merkmal $m_i$ eine $1$ bei dem das $i$ für den Wert $y$ steht. Alle anderen neuen Merkmale $m_j$ des Datenpunkts tragen den Wert $0$.
Man fasst also die Werte des ursprünglichen Merkmals als eigenständige Merkmale auf und vermerkt für jeden Datenpunkt om der Wert vorliegt oder nicht.

In unserem DataFrame überführen wir nur die kategorialen Variablen, die 10 oder wenige Werte besitzen.
Alle anderen werden wir später verwerfen.
Um die One-Hot-Codierung einer DataFrame Spalte zu erzeugen, verwenden wir hier die Pandas-Funktion `get_dummies`.

In [ ]:
df_num = df_golf6
for c in df.select_dtypes(include=['object']).columns:
    print(c, df_num[c].nunique())
    if df_num[c].nunique()<10:
        df_num = pd.get_dummies(df_num, columns = [c])

In [ ]:
df_num.info()

In [ ]:
df_num.drop(df_num.select_dtypes(include=['object']).columns, axis='columns', inplace=True)
df_num.info()


## Daten Visualisieren und mit Statistischen Kennwerten beschreiben

Als Nächstes möchten wir einen Eindruck erhalten, wie die einzelnen Merkmale miteinander zusammenhängen.
Ist die Anzahl der Merkmale überschaubar, bietet es sich an, **Streudiagramme** (engl. *scatter plot*) für alle Paare von Merkmalen zu zeichnen.

Streudiagramm werden die Elemente zweier Datenreihen paarweise (als X- und Y-Werte) in ein kartesisches Koordinatensystem eingetragen.
Dadurch ergibt sich eine sogenannte Punktwolke, die anzeigt, in welcher Abhängigkeit die beiden Datenreihen zueinander stehen.

Für Python gibt es verschiedene Bibliotheken, mit denen man Streudiagramme zeichnen kann.
Da unsere Daten als Pandas `DataFrame` vorliegen, bietet sich in unserem Fall die Funktion `scatter_matrix` aus dem Paket `pandas.plotting` an.
Da wir in ein kartesisches Koordinatensystem nur numerische Koordinaten eintragen können, reduzieren wir den DataFrame auf Spalten, die entweder als 64-bit `float` oder `int` Typen vorliegen.

In [ ]:
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

# Wir selektieren einige der Spalten unseres Dataframes...
df_scatter = df_num[['preis', 'karosserieform_Cabrio', 'erstzulassung_jahr','kilometer', 'leistung', 'tueren']]
# ...und erzeugen die Scatter Plots
splot = scatter_matrix(df_scatter, figsize=(10,10));

Die Abbildung zeigt nun eine Vielzahl von *Punktwolken*.
Uns interessiert hier vor allem die erste Zeile (oder die erste Spalte, da die Graphen an der Hauptdiagonalen gespiegelt sind).
Dort sind die Punktwolken abgebildet, bei denen die eine Datenreihe der Preis des Autos ist.
Wir wollen später eine **Schätzfunktion** trainieren, die möglichst gut den Preis eines Gebrauchtfahrzeugs vorhersagen kann.
Daher müssen wir herausfinden, mit welchen anderen Merkmalen der Preis möglichst eng linear *korreliert*.
Grafisch betrachtet, suchen wir nach Punktwolken, die sich möglichst eng anhand **einer** Geraden orientieren.

Erfreulicherweise sind wir nicht auf die visuelle Analyse der Daten angewiesen.
Der **Korrelationskoeffizient** $r$ ist eine Kenngröße, die angibt, wie stark die Werte zweier Datenreihen $x$ und $y$ miteinander linear korrelieren.
Der Wert von $r$ liegt immer in der Grenzen $[-1,1]$.
Dabei bedeutet ein sehr niedriger (etwa $r<-0.8$) oder ein sehr hoher Wert (etwa $r>0.8$), dass die Reihen korrelieren, es also einen statistischen Zusammenhang der Datenreihen gibt.
Ein Wert von $r$ um den Nullpunkt (etwa $-0.5<r<0.5$) bedeutet, dass es keinen signifikanten Zusammenhang gibt.

Für die Berechnung der **Korrelationsmatrix** mit der Pandas Funktion `corr()`, nehmen wir den Datensatz, der alle Merkmale enthält.
Aus der berechneten Matrix selektieren wir die Spalte *Preis* und geben die Korrelationskoeffizienten in absteigend sortierter Reihenfolge aus:

In [ ]:
# Wir delektieren aus dem DataFrame df_num nur die "numerischen" Spalten
df_select = df_num.select_dtypes(include=np.number)
corr_matrix = df_select.corr()
corr_matrix["preis"].sort_values(ascending=False)

Da für uns die absolut größten Werte interessant sind, wenden wir die Betragsfunktion `abs()` auf die Werte an und geben die sortierte Liste erneut aus.

In [ ]:
corr_matrix["preis"].abs().sort_values(ascending=False).head(10)

**Aufgabe: Berechnen Sie den Korrelationskoeffizienten $\rho$ für die Spalten `preis` und `kilometer` "per Hand"**

Ziehen Sie dazu zuerst aus dem `DataFrame` `temp_df` mit dem Attribut `values` die Werte der (bereinigten) Datenreihen in NumPy Arrays.
Sie können zur Berechnung die NumPy Funktionen `mean()` (für den Mittelwert, bzw. Erwartungswert $\mu$) und `std()` (für die Standardabweichung $\sigma$) benutzen

$ \rho(X,Y) = \Large \frac{cov(X,Y)}{\sigma_X\sigma_Y}$  $\,$   mit  $\,$   $ \large cov(X,Y)=E\left[ (X-\mu_X) (Y-\mu_Y) \right]$

In [ ]:
import numpy as np

def cov_xy(x, y):
    ''' Berechne die Kovarianz cov(x,y)
        der beiden Merkmale x und y
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Cell
#----------

test_x0 = np.array([  1.,  -4.,   7.,  -6., -14.,  -1.,  -3., -33.,   6.,   3.])
test_y0 = np.array([  2.,  -7., -14.,   3.,  -9.,  19.,  14.,  -3., -11., -23.])

assert cov_xy(test_x0, test_y0) == -18.46

In [ ]:
def calc_r(x, y):
    ''' Berechne den Korrelationskoeffizienten 
        roh(x, y) der beiden Merkmale x und y
    '''
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
# Test Cell
#----------

assert np.around(calc_r(test_x0, test_y0), 4) == -0.1362

Nun können wir die Funktion `calc_r` mit unserem Datensatz ausprobieren:

In [ ]:
temp_df = df.query("modell=='Golf VI'")[["preis","kilometer"]].dropna()
ty = temp_df["preis"].values
tx = temp_df["kilometer"].values
calc_r(tx,ty)

NumPy stellt ebenfalls eine Funktion `corrcoef()` zur Verfügung.
Diese stellt die Ergebnisse als Korrelationsmatrix dar, wir können sie aber ungeachtet dessen verwenden, um unser Ergebnis zu überprüfen:

In [ ]:
np.corrcoef(tx,ty)

## Exkurs: Data Science mit Orange3

Wir haben nun erste Einblicke in unseren Datensatz erhalten und werden in den kommenden Schritten ein lineares Modell trainieren, mit dem wir Preisvorhersagen treffen können.
Diese weiteren Schritte werden wir mit ebenfalls mit Python, bzw. mit den zugehörigen Bibliotheken *Pandas* und *Scikit-learn* durführen.

An dieser Stelle machen wir allerdings einen kurzen Exkurs in die Datenanalyse mit grafischen Werkzeugen.
Dazu verwenden wir das Werkzeug **Orange3**.

Orange ist ein grafisches open-source Werkzeug für Data Science und Machine Learning Anwendungen.
Es kann als eigenständige Anwendung auf Windows, Linux oder MacOS installiert werden, oder (wie in unserem Fall) als Erweiterung einer bestehenden Python Installation.
Orange funktioniert nach dem Baukasten-Prinzip: Das Framework beinhaltet eine Reihe von Modulen für verschiedene Aufgaben, die je nach Anwendung zu einer komplexen Data Mining Pipeline zusammengeschaltet werden können.
Gleichzeitig ist Orange erweiterbar.
So lassen sich mit recht einfachen Mitteln eigene Module in Python entwickeln und in den Baukasten integrieren.

Um unsere Auto-Daten in Orange3 weiter zu verarbeiten, exportieren wir den relevanten Datensatz in eine CSV-Datei.
CSV steht für **C**omma-**S**eparated **V**alues und ist ein Standardformat um tabellarische Daten textuell zu speichern.

In [ ]:
df_select.to_csv("golf6.csv")

Dieses Daten können wir nun ein einem Neuen Orange3 **Workflow** einbinden.

**Aufgabe: Analysieren Sie den Datensatz in Orange3 mithilfe eines "Linear Regression" Modells.**
    
Gehen Sie in folgenden Schritten vor:
  1. Öffnen Sie Orange3 (über Ihr Programm-Menü oder den *Anaconda Navigator*).
  1. Wählen Sie im Welcome-Menü die Option *New Workflow*.
  1. Wählen Sie aus dem Katalog *Data* das Modul *CSV File Import*. In den Optionen des Moduls wählen Sie die Datei "*golf6.csv*" aus.
  1. Erstellen Sie eine Data Table, um den Inhalt der CSV-Datei zu betrachten.
  1. Erstellen Sie einen *Scatter Plot* aus dem Bereich *Visualize* des Katalogs. Plotten Sie die *Kilometer* (x-Achse) gegen den *Preis* (y-Achse).
  1. Fügen Sie ein Modul des Typs *Select Columns* ein. Wählen Sie aus den Daten die Merkmale `karosserieform_Cabrio`, `erstzulassung_jahr`, `kilometer` und `leistung` als *Features* aus, das Merkmal `preis` soll die *Target Variable* sein.
  1. Fügen Sie ein Modul des Typs *Data Sampler* ein. 70% der Daten sollen als *Data Sample* ausgewählt werden, die restlichen 30% werden als *Remaining Data* bezeichnet.
  1. Fügen Sie aus dem Bereich *Model* ein Modul des Typs *Linear Regression* ein und Trainieren Sie das Modell mit dem *Data Sample*.
  1. Fügen Sie ein *Predictions* Modul (aus dem Bereich *Evaluate*) ein und verbinden SIe es mit dem *Linear Regression* Modell. Auf den Eingang *Data* legen Sie den Ausgang *Remaining Data* des *Data Sampler* Moduls.
  1. Ihr *Workflow* sollte nun in etwa wie auf der Abbildung unten aussehen. Zu welchem Ergebnis führt das Modell?

![](https://github.com/fh-swf-hgi/ml_aki/raw/main/u2/orange3-car1a.png)

**Aufgabe: Vergleichen Sie das "Linear Regression" Modell mit einem "Regression Tree". Verwenden Sie dazu das "Test and Score" Modul wie auf der Abbildung unten. Welches Modell liefert die besseren Ergebnisse?**

![](https://github.com/fh-swf-hgi/ml_aki/raw/main/u2/orange3-car2a.png)

## Daten selektieren und aufbereiten
Wir wollen uns für die weitere Analyse auf die Merkmale *karosserieform_Cabrio*, *erstzulassung_jahr*, *kilometer* und *leistung* konzentrieren, da sie die höchste Korrelation mit dem Merkmal *preis* aufweisen.

In [ ]:
golf_6_num = df_select[["preis","karosserieform_Cabrio","erstzulassung_jahr","kilometer","leistung"]]
golf_6_num.info()

Wenn wir den Datensatz auf diese Merkmale reduzieren, fällt auf, dass nicht alle Datenreihen die gleiche Anzahl an Werten beinhalten (siehe Spalte `Non-Null`).
Diese "Lücken" im Datensatz sind kritisch für die weiteren Schritte.
Viele Algorithmen kommen mit unvollständigen Daten, bzw `NaN` Elementen nicht zurecht.
Daher eliminieren wir die Datenpunkte, die ein `NaN` enthalten mit der Funktion `dropna()` aus dem Datensatz.

In [ ]:
golf_6_num = golf_6_num.dropna()
golf_6_num.info()

Wenn wir für die ausgewählten Merkmale Streudiagramme plotten, sehen wir, dass sich für den Preis Abhängigkeiten der Art **"je mehr, desto mehr"** (*erstzulassung_jahr*, *leistung*) sowie **"je mehr, desto weniger"** (*kilometer*) abzeichnen.

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(golf_6_num, figsize=(10, 8));

Als nächstes wollen wir den Datensatz in 2 Teile aufteilen.
Einen **Trainingsdatensatz** sowie einen **Testdatensatz**.
Mit dem ersten *trainieren* wir unser Modell.
Mit dem zweiten Datensatz testen wir die trainierte Modellfunktion.
Mit diesem Ansatz adressieren wir das Problem des **Overfittings** (Überanpassung).
Es kann beim Trainieren des Modells dazu kommen, dass der Lern-Algorithmus die Modell-Parameter zu sehr auf den Trainingsdatensatz anpasst.
Durch vorhalten eines separaten Datensatzes, der nicht zum Trainieren benutzt wird, kann dieses Problem erkannt werden.

Es ist üblich, etwa 1/5 bis 1/3 der Daten für den Trainingsdatensatz und die restlichen Daten für den Testdatensatz zu verwenden.
Um die Daten aufzuteilen verwenden wir die Funktion `train_test_split` aus dem *sklearn* Modul `model_selection`.
Mit dem Parameter `test_size` bestimmen wir den Anteil der Testdaten. Mit `random_state` wird der Zufallszahlengenerator initialisiert, über den die Datenpunkte ausgewählt werden. Legt man `random_state` fest, so wird immer die gleiche Aufteilung vorgenommen. Dies hat den Vorteil, dass die Analysen vergleichbar sind.

In [ ]:
from sklearn.model_selection import train_test_split

y = golf_6_num['preis']
# Wir wählen nur die beiden aussagekäftigsten Merkmale aus
X = golf_6_num[["karosserieform_Cabrio","erstzulassung_jahr"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Lineare Regression

Bisher haben wir ausschließlich den Datensatz grafisch und statistisch analysiert, aber noch kein Machine Learning Algorithmus angewendet.
Wir sind jetzt bereit, aus den gesammelten Daten eine *Schätzfunktion* abzuleiten.
Diese Funktion soll auf Grundlage eines oder mehrerer Merkmale eine Schätzung für den Preis des Fahrzeugs ausgeben.
Diese Schätzung soll so ausfallen, dass sie möglichst gut zu den bestehenden Datenpunkten passt.
damit könnte man z.B. der *Richtpreis* für eine neue Annonce berechnen oder bestimmen, ob ein Angebot im Vergleich eher teuer oder günstig ist.
Um diese Schätzfunktion zu entwickeln, stürzen wir uns auf die Zusammenhänge, die wir in der Analysephase herausgearbeitet haben und trainieren auf dieser Grundlage ein lineares Regressionsmodell.

Es ist durchaus möglich, das Regressionsmodell mit den Mitteln, die wir bisher kennengelernt haben (NumPy, Pandas), aufzustellen und zu trainieren.
Um die Trainingsalgorithmen kennenzulernen, werden wir das zu einem späteren Zeitpunkt auch tun.
Allerdings ist es in der Praxis eher unüblich, Machine Learning (ML) Modelle "per Hand" zu entwickeln.
Es gibt, insbesondere für Python, eine Vielzahl von Bibliotheken, mit denen ML Modelle effizient trainiert werden können.

Eine weit verbreitete ML Bibliothek für Python ist **Scikit-Learn** (oder auch *sklearn*).
Eine Besonderheit von Scikit-Learn ist, dass die Bibliothek sehr umfassend ist und ein breites Spektrum von ML-Algorithmen unterstützt.
Darunter sind Algorithmen zur Klassifikation, Regression und für das Clustering.
Auch unterstützende Methode wie etwa Algorithmen zur Dimensionsreduktion, zur Modellauswahl und zur Vorverarbeitung der Rohdaten sind in der Bibliothek enthalten.

Die Funktionen zur linearen Regression entnehmen wir aus dem Modul `sklearn.linear_model`.
Zur besseren Übersicht, weisen wir die abhängige Variable auf `y` (bzw. `y_test`) zu, die unabhängige Variable auf `X` (bzw. `X_test`)

Wir erzeugen mit `lin_reg` ein Modell als Objekt der Klasse `LinearRegression` und trainieren das Modell mit dem Aufruf `fit(X,y)` auf unsere Daten.

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression().fit(X_train, y_train)

Nun haben wir das Modell trainiert, aber wie gut passt es zu unserem Datensatz?
Eine Antwort aus diese Frage kann uns der durchschnittlichen Fehler für unseren Trainingsdatensatz geben.
Die Funktion `mean_absolute_error` berechnet den Mittelwert der absoluten Fehler für die Schätzung.

Damit wir mehrere Modelle schneller untersuchen können, schreiben wir eine Funktion `eval_model` die auf Grundlage der Testdaten (`X_test`) sowie der zugehörigen tatsächlichen Ergebnisse der Testdaten (`y_test`) einen *Schätzer* (`model`) untersucht.

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

def eval_model(model, X_test, y_test):
    avg_price = y_test.values.mean()
    print("Mittlerer Preis: %d EUR" % avg_price)
    pred = model.predict(X_test)
    pred_err = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    pred_err_pc = pred_err*100//avg_price
    print(f"Im Mittel liegt die Preisschätzung um {pred_err:.2f}€ oder {pred_err_pc:.1f}% daneben (R²={r2:.3f})")
    
    

Nun rufen wir `eval_model` mit unserem Modell `lin_reg` auf:

In [ ]:
eval_model(lin_reg, X_test, y_test)

Eine mehr als 10%-ige Abweichung von Preis ist nicht unbedingt gut.
Wir können das Ergebnis verbessern, indem wir mehr Variablen in das Modell aufnehmen.

**Aufgabe: Trainieren Sie ein LinearRegression-Modell mit allen Merkmalen des Datensatzes `golf_6_num`. Wie gut kann das Modell den erwarteten Preis vorhersagen?**


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Aufgabe: Verwenden Sie weitere Regressionsverfahren um die Ergebnisse mit dem Linear-Regression-Modell zu vergleichen**

*sklearn* bietet noch weitere Klassen für Regressionsanalysen. Darunter etwa `sklearn.linear_model.ElasticNet`, `sklearn.ensemble.GradientBoostingRegressor` und `sklearn.tree.DecisionTreeRegressor`.
Verwenden Sie diese Modelle und vergleichen Sie die Ergebnisse mit dem Modell oben zu vergleichen. Welches Verfahren liefert die besten Ergebnisse?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()